In [1]:
import collections
import random

In [39]:
SUPERTAGS_FILEPATH = '../data/supertags.txt'
BIGRAMS_FILEPATH = '../data/poleval_2grams.txt'
PAN_TADEUSZ_FILEPATH = '../data/pan-tadeusz.txt'

In [3]:
TAGS = {}
with open(SUPERTAGS_FILEPATH) as f:
    for line in f:
        word, tag = line.split()
        TAGS[word] = tag

In [4]:
def tag_of(word):
    word_ = word.lower()
    return TAGS.get(word_, f'^{word_}'[-3:])

# 3

In [6]:
# unigram stats, doubled but it doesnt matter
UNIGRAMS_BY_TAG = collections.defaultdict(lambda: collections.defaultdict(int))
with open(BIGRAMS_FILEPATH) as f:
    for line in f:
        count_raw, word1, word2 = line.split()
        tag1 = tag_of(word1)
        tag2 = tag_of(word2)
        count = int(count_raw)
        UNIGRAMS_BY_TAG[tag1][word1] += count
        UNIGRAMS_BY_TAG[tag2][word2] += count

In [18]:
def gen_ug(word):
    """Generate a similar word using unigrams"""
    tag = tag_of(word)
    ug = UNIGRAMS_BY_TAG[tag]
    pop, w = list(ug.keys()), list(ug.values())
    return random.choices(pop, w)[0]

In [15]:
def generate_by_tags(text):
    """Input must be tokenized"""
    words = text.split()
    res = (gen_ug(word) for word in words)
    return ' '.join(res)

In [24]:
EXAMPLES = """
Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę .
Dwa razy ostra baranina na grubym !
Litwo , ojczyzno moja , Ty jesteś jak zdrowie
Zapraszam wszystkich do wrzucania swoich implementacji w różnych językach w komentarzach .
Szanuję twórcę za talent , ale nawet się nie uśmiechnąłem
Mam takie skromne marzenie dotyczące uniwersum Harrego Pottera
Eksperymenty przeprowadza się na studentach , bo jest ich dużo i na szczurach , bo są inteligentne .
""".split('\n')[1:-1]

In [25]:
for example in EXAMPLES:
    print(generate_by_tags(example))

śląski paweł zbadał do nieoficjalnej sprawie jeśli cytowaną inflację .
dwa dni indywidualna firma i następnym !
warto , kolo taka , ty jesteś jak ostrze
spodziewam których w wyposażenia niektórych noweli w sejmowych zakładach dla krajach .
informuję pracodawcę w druk , się trzeba dla nie spróbowałem
mam które fantazyjne morze odpowiadające prezydium mikroostrożnościowego infoafera
hotele zajmuje na w podatnikach , przez jest ich na o i bezkręgowcach , się są publiczne .


# 4

In [11]:
# bigram stats
BIGRAMS_BY_TAG_BY_PREV = collections.defaultdict(
    lambda: collections.defaultdict(
        lambda: collections.defaultdict(int)
    )
)
with open(BIGRAMS_FILEPATH) as f:
    for line in f:
        count_raw, word1, word2 = line.split()
        count = int(count_raw)
        tag2 = tag_of(word2)
        BIGRAMS_BY_TAG_BY_PREV[word1][tag2][word2] += count

In [19]:
def gen_bg(prev, word):
    """Generate a similar word using bigrams,
    prev is previously generated here, not source
    """
    tag = tag_of(word)
    bgs = BIGRAMS_BY_TAG_BY_PREV[prev][tag]
    if not bgs:
        return None
    pop, w = list(bgs.keys()), list(bgs.values())
    return random.choices(pop, w)[0]

In [22]:
def generate_by_tags2(text):
    """Input must be tokenized"""
    words = text.split()
    word = words.pop(0)
    # generate first by unigram
    res = [gen_ug(word)]
    for word in words:
        nxt = gen_bg(res[-1], word) or f'| {gen_ug(word)}'
        res.append(nxt)
    return ' '.join(res)

In [29]:
for example in EXAMPLES:
    print(generate_by_tags2(example))

zgodny kumpel zakupił w niewłaściwej obsłudze i przemyślaną decyzję .
dwa razy można kilka wcześniej wyjazdowym | !
sporo , warto ta , ty jesteś jak dokowanie
informuję wszystkich z morza | niektórych | pracy | w | pielęgniarskich | lasach | i | aktach | .
wyjaśniam | rzeczoznawcę | w | program | , | poprzez | że | przecież | nie | przyjąłem
mam takie mądre bieganie | dotyczące | centrum | antyawarskiego | altera
zawody organizuje się na polakach , z jest ich o delhi w sokołach , że są rzetelne .


# 6

In [51]:
LETTERS_PL = set('aąbcćdeęfghijklłmnńoópqrsśtuvwxyzżź')
VOWELS_PL = set('aeyioąęuó')

In [63]:
def sylc_in(word_raw):
    word_ = f'{word_raw}^'
    sylc = 0
    for i in range(len(word_raw)):
        # I know, I know
        if word_[i] == 'i' and word_[i + 1] in VOWELS_PL:
            pass
        elif word_[i] in VOWELS_PL:
            sylc += 1
    return sylc

In [64]:
def count_syllables(text_raw):
    words_raw = text_raw.lower().split()
    res = []
    for word_raw in words_raw:
        res.append(sylc_in(word_raw))
    return tuple(res)

In [61]:
def taginate(text_raw):
    buf = []
    tokens = []
    for ch in text_raw.lower():
        if ch in LETTERS_PL:
            buf.append(ch)
        elif buf:
            tokens.append(''.join(buf))
            buf.clear()
    return tuple(tag_of(token) for token in tokens)

In [62]:
PT_SYLLABLE_COUNTS = set()
PT_TAG_SEQS = set()
with open(PAN_TADEUSZ_FILEPATH) as f:
    for line in f:
        sylc = count_syllables(line)
        tags = taginate(line)
        PT_SYLLABLE_COUNTS.add(sylc)
        PT_TAG_SEQS.add(tags)

In [102]:
def _next(sylc, tag, rhyme=''):
    ug = UNIGRAMS_BY_TAG[tag]
    words = [k for k in ug if k.endswith(rhyme) and sylc_in(k) == sylc]
    if not words:
        return None
    w = [ug[k] for k in words]
    return random.choices(words, w)[0]

def gen_pt_line(rhyme='', *, max_iter=10_000):
    """Try random tags and syllable counts
    until a line is generated. Generation is
    done in reverse order for rhyming performance.
    """
    iterc = 0
    tag_opts = list(PT_TAG_SEQS.copy())
    random.shuffle(tag_opts)
    for tags_line in tag_opts:
        sylc_opts = [
            scl for scl in PT_SYLLABLE_COUNTS
            if len(scl) == len(tags_line)
        ]
        random.shuffle(sylc_opts)
        for sylc_line in sylc_opts:
            iterc += 1
            if iterc > max_iter:
                return None
            buf = []
            last = len(sylc_line) - 1
            for i, (sylc, tag) in enumerate(zip(
                reversed(sylc_line), reversed(tags_line)
            )):
                rhm = rhyme if i == 0 else ''
                nxt = _next(sylc, tag, rhm)
                if nxt is None:
                    break
                buf.append(nxt)
                if i == last:
                    return ' '.join(reversed(buf))
    return None

In [119]:
def get_rhyme(line):
    """Get line ending to last-but-one vowel"""
    buf = []
    lbo = True
    line_ = f'^{line}'
    for i in range(len(line), -1, -1):
        buf.append(line_[i])
        if line_[i] in VOWELS_PL:
            if line_[i - 1] == 'i':
                continue
            if not lbo:
                return ''.join(reversed(buf))
            lbo = False

def gen_pt_diverse(max_tries=100, max_line_iter=5000):
    for _ in range(max_tries):
        fst = gen_pt_line(max_iter=max_line_iter)
        if fst is None:
            break
        rhyme = get_rhyme(fst)
        snd = gen_pt_line(rhyme, max_iter=max_line_iter)
        if snd:
            return fst, snd
    return None

In [123]:
for _ in range(20):
    dv = gen_pt_diverse()
    if dv:
        print('\n'.join(dv))

w mnie żyje mdły niektóry szwank ocena jaki
ale odpowiednią swą mszą wówczas rybaki
nej nich miękkości zawarte orderem
samce aby w hałd albo dnia wręczyli erem
jako goście najmu na oleju dostali
że mimo co za wejściu mam i powiem zali
woż zdał wroński pochodził to temat ostatni
tutaj właśnie i chcemy ktoś kogo uzdatni
wieść do miejscu przy grze na do sprzedaży jana
pije nie wyłącznie trio się a grę tytana
później oraz czy podjął praw butów zasiadał
mogę kto 100-latków śmierci mszę wyspowiadał
bardziej trendami i skracano ręce
się świecie domuchowski według tak książęce
i tylko habbaniji łza świętego pana
a grając z odnotowuję internet wymiana
cele tańczą także pan ich realizuje
za zostało do weszli więc cny tu pracuje
się nim wieszczył pan poseł oraz w apollonie
porucznik wyniósł zwierzaka tytonie
z znalazłem określenia według gry waszeci
ktokolwiek niej nie zleci
zamieszkała tri była w spadła zatrudniona
przez ciasno bez tych upraw miłością nie śledziona
ambasadą akcje jak złem trwać 